In [1]:
import sys
import argparse
import telnetlib
import time
import json
from datetime import datetime

def connect(host="",user="",password=""):
    tn = telnetlib.Telnet(host,4000)
    tn.read_until(b">")

    if password != "":
        s = "auth " + user + " " + password + "\n"
        tn.write(bytes(s,'ascii'))
        r = tn_wait(tn)
        print(r.decode('ascii'))   

    return tn

def tn_wait(tn):
    r = ''
    t = 0
    while len(r) < 2 and t < 10:
        r = tn.read_very_eager()
        time.sleep(1)
        t=t+1
    return r

def save_downloaded(already_downloaded):
    with open('already_downloaded.json', 'w',encoding="ascii", newline='\r\n') as outfile:
        json.dump(already_downloaded, outfile, indent=2, ensure_ascii=False)

def read_downloaded():
    already_downloaded = []
    with open('already_downloaded.json') as json_file:
        already_downloaded = json.load(json_file)
    return already_downloaded

def add_downloaded(new_download):
    already_downloaded = read_downloaded()
    already_downloaded = already_downloaded + new_download
    already_downloaded = list(set(already_downloaded))
    already_downloaded.sort()
    save_downloaded(already_downloaded)
    
def check_downloaded(i):
    already_downloaded = read_downloaded()
    if i.find(b"ed2k")>0:
        b = i.split(b":")[2].decode('ascii')
        if b in already_downloaded:
            return True
    return False

def choose_download(r):
    # TODO: filter by filename length (optionally)
    r_clean = [i for i in r.splitlines() if i.find(b"ed2k")>0]
    return [int(i[i.find(b"m[")+2:i.find(b"]")]) for i in r_clean if not(check_downloaded(i))]

def donwload_resuts(tn,r,max_download=0):
    new_download = [i.split(b":")[2].decode('ascii') for i in r.split() if i.find(b"ed2k")>0]
    res = choose_download(r)
    c=0
    for i in res:
        d = "d %d\n" % i
        tn.write(bytes(d,'ascii'))
        r = tn_wait(tn)
        # print(r.decode('ascii'))
        c = c + 1
        if max_download>0 and max_download<=c:
            pass
    add_downloaded(new_download)

def single_search(tn, search,max_download=0):
    tn.write(bytes(search,'ascii'))
    r = tn_wait(tn)
    #print(r.decode('ascii'))

    time.sleep(30)

    tn.write(bytes("vr\n",'ascii'))
    r = tn_wait(tn)
    #print(r.decode('ascii'))

    if r[0:70].decode('ascii').find("\n0 results") == -1:
        # print(r[0:70].decode('ascii')[0:70])
        donwload_resuts(tn,r,max_download)
        
    
def pattern_search(tn, pattern,init_sq,end_sq,max_download=0):
    for j in range(init_sq,end_sq):
        print(j)
        search = pattern % j
        single_search(tn,search,max_download)  
        
def raw_search(ext):
    for e in ext.split(' '):
        print(e)
        single_search(tn,"s DSC {}\n".format(e))
        single_search(tn,"s IMG {}\n".format(e))
        
def cancel_pattern(tn,pattern):
    tn.write(bytes("vd\n",'ascii'))
    r = tn_wait(tn)
    jpgs = [int(i[i.find(b"m[")+3:i.find(b"]")]) for i in r.splitlines() if i.find(pattern)>0]
    s = "cancel"
    for j in jpgs:
        s = s + " " + str(j)
    s = s + " \n"
    tn.write(bytes(s,'ascii'))
    r = tn_wait(tn)
    tn.write(bytes("confirm yes\n",'ascii'))
    r = tn_wait(tn)
    
   

host = "192.168.2.35"
user = "admin"
password = ""

t_init = datetime.now()

tn = connect(host,user,password)

# RAWs

In [2]:
raw_search('.3fr') # (Hasselblad)
raw_search('.ari') # (Arri Alexa)
raw_search('.arw .srf .sr2') # (Sony)
raw_search('.bay') # (Casio)
raw_search('.braw') # (Blackmagic Design)
raw_search('.cri') # (Cintel)
raw_search('.crw .cr2 .cr3') # (Canon)
raw_search('.cap .iiq .eip') # (Phase_One)
raw_search('.dcs .dcr .drf .k25 .kdc') # (Kodak)
raw_search('.dng') # (Adobe)
raw_search('.erf') # (Epson)
raw_search('.fff') # (Imacon/Hasselblad raw)
raw_search('.gpr') # (GoPro)
raw_search('.mef') # (Mamiya)
raw_search('.mdc') # (Minolta, Agfa)
raw_search('.mos') # (Leaf)
raw_search('.mrw') # (Minolta, Konica Minolta)
raw_search('.nef .nrw') # (Nikon)
raw_search('.orf') # (Olympus)
raw_search('.pef .ptx') # (Pentax)
raw_search('.pxn') # (Logitech)
raw_search('.R3D') # (RED Digital Cinema)
raw_search('.raf') # (Fuji)
raw_search('.raw .rw2') # (Panasonic)
raw_search('.raw .rwl .dng') # (Leica)
raw_search('.rwz') # (Rawzor)
raw_search('.srw') # (Samsung)
raw_search('.x3f') # (Sigma)

.3fr
.ari
.arw
.srf
.sr2
.bay
.braw
.cri
.crw
.cr2
.cr3
.cap
.iiq
.eip
.dcs
.dcr
.drf
.k25
.kdc
.dng
.erf
.fff
.gpr
.mef
.mdc
.mos
.mrw
.nef
.nrw
.orf
.pef
.ptx
.pxn
.R3D
.raf
.raw
.rw2
.raw
.rwl
.dng
.rwz
.srw
.x3f


# CVs

In [3]:
single_search(tn,'s CV DOCX\n')

In [4]:
single_search(tn,'s Curriculum DOCX\n')

# .JPG pattern

In [5]:
# pattern_search(tn,"s IMG_%04d.JPG -minsize 1500000 -maxsize 15000000 \n",1,9999)

In [6]:
single_search(tn,'s IMG 2501 JPG -minsize 1500000 -maxsize 15000000 \n')

# Clean

In [7]:
# cancel_pattern(tn,b'jpg')  
cancel_pattern(tn,b'oraya')  
cancel_pattern(tn,b'.avi')  

# Time

In [8]:
t_end = datetime.now()


t_exec = (t_end - t_init)
t_exec

datetime.timedelta(seconds=3145, microseconds=111955)

In [9]:
# single_search(tn,'s bianca pose\n') # :( mierda!!!!

In [3]:
cancel_pattern(tn,b'jpg')  